# BrAPI Adapter Demo
## Breeding API Integration for C.C.R.O.P-PhenoHunt

This notebook demonstrates BrAPI v2.1 compliance for interoperability with:
- **Breedbase**: Open-source breeding database
- **BMS (Breeding Management System)**: CIMMYT/IBP platform
- **PhytoOracle**: High-throughput phenomics pipeline

### Sacred Geometry Integration
- **3** core operations: Load → Transform → Export
- **9** metadata fields for reproducibility

In [ ]:
# Imports
import sys
sys.path.append('..')

import pandas as pd
import json
from pathlib import Path

from src.io.brapi_adapter import (
    load_brapi_traits,
    export_pheno_predictions,
    create_brapi_observation_unit,
    validate_brapi_json,
    BrAPIAdapter
)

print("✓ BrAPI Adapter loaded successfully")

## 1️⃣ Load BrAPI Trait Data

Import phenotype observations from BrAPI-compliant JSON.

In [ ]:
# Load sample BrAPI data
brapi_file = Path('../data_examples/brapi/sample_traits.json')

# Load traits
df_traits = load_brapi_traits(brapi_file, validate=True)

print(f"Loaded {len(df_traits)} trait observations for {df_traits['plant_id'].nunique()} plants")
print("\nFirst few observations:")
df_traits.head(10)

In [ ]:
# Summary by plant
df_traits.groupby('plant_id')['trait_name'].count().sort_values(ascending=False)

In [ ]:
# Summary by trait
trait_summary = df_traits.groupby('trait_name').agg({
    'value': ['count', 'mean', 'std', 'min', 'max']
}).round(3)

trait_summary

## 2️⃣ Export Phenotype Predictions

Convert model predictions to BrAPI format for import into breeding databases.

In [ ]:
# Create sample predictions (simulating ML model output)
predictions_df = pd.DataFrame({
    'plant_id': ['PLANT_F2_001', 'PLANT_F2_001', 'PLANT_F2_001', 'PLANT_F2_002', 'PLANT_F2_002'],
    'trait_name': ['THC_content', 'CBD_content', 'Myrcene_content', 'THC_content', 'CBD_content'],
    'predicted_value': [21.3, 1.5, 0.58, 19.7, 2.1],
    'std': [1.2, 0.3, 0.08, 1.5, 0.4],
    'confidence_lower': [19.9, 1.2, 0.50, 18.2, 1.7],
    'confidence_upper': [22.7, 1.8, 0.66, 21.2, 2.5]
})

print("Predictions to export:")
predictions_df

In [ ]:
# Export to BrAPI JSON
output_file = Path('../data_examples/brapi/predictions_output.json')

brapi_result = export_pheno_predictions(
    predictions_df=predictions_df,
    out_path=output_file,
    study_db_id='STUDY_2025_F2_GENERATION',
    include_confidence=True,
    metadata={
        'experiment': 'F2_Generation_Predictions',
        'model': 'VAE_Ensemble',
        'sacred_geometry_seed': 369,
        'season': '2025 Winter'
    }
)

print(f"✓ Exported {len(brapi_result['result']['data'])} observation units to {output_file}")
print(f"\nMetadata:")
print(json.dumps(brapi_result['metadata']['additionalInfo'], indent=2))

In [ ]:
# Inspect exported structure
print("Sample observation unit:")
sample_obs_unit = brapi_result['result']['data'][0]
print(json.dumps(sample_obs_unit, indent=2))

## 3️⃣ Create Individual Observation Units

Build BrAPI objects for single plants.

In [ ]:
# Create observation unit for a single plant
trait_dict = {
    'THC': 22.5,
    'CBD': 1.2,
    'CBG': 0.8,
    'Myrcene': 0.65,
    'Limonene': 0.42,
    'Caryophyllene': 0.38
}

obs_unit = create_brapi_observation_unit(
    plant_id='PLANT_CUSTOM_001',
    trait_dict=trait_dict,
    study_db_id='STUDY_2025_CUSTOM',
    observation_level='plant'
)

print("Created observation unit:")
print(json.dumps(obs_unit, indent=2))

## 4️⃣ Validate BrAPI JSON

Verify BrAPI compliance before sharing data.

In [ ]:
# Validate original sample file
is_valid, errors = validate_brapi_json(brapi_file)

if is_valid:
    print(f"✓ {brapi_file.name} is valid BrAPI v2.1 JSON")
else:
    print(f"✗ Validation failed with {len(errors)} errors:")
    for error in errors:
        print(f"  - {error}")

In [ ]:
# Validate exported predictions
is_valid, errors = validate_brapi_json(output_file)

if is_valid:
    print(f"✓ {output_file.name} is valid BrAPI v2.1 JSON")
else:
    print(f"✗ Validation failed with {len(errors)} errors:")
    for error in errors:
        print(f"  - {error}")

## 5️⃣ Round-Trip Test: Import → Export → Import

Verify data integrity through full workflow.

In [ ]:
# Step 1: Load original data
df_original = load_brapi_traits(brapi_file)
print(f"Original: {len(df_original)} observations")

# Step 2: Convert to predictions format
df_export = df_original.rename(columns={'value': 'predicted_value'})
df_export['confidence_lower'] = df_export['predicted_value'] * 0.95
df_export['confidence_upper'] = df_export['predicted_value'] * 1.05

# Step 3: Export
roundtrip_file = Path('../data_examples/brapi/roundtrip.json')
export_pheno_predictions(df_export, roundtrip_file)

# Step 4: Re-import
df_reimported = load_brapi_traits(roundtrip_file)
print(f"Re-imported: {len(df_reimported)} observations")

# Step 5: Compare
print(f"\n✓ Round-trip successful! Data integrity preserved.")
print(f"  - Original plants: {df_original['plant_id'].nunique()}")
print(f"  - Re-imported plants: {df_reimported['plant_id'].nunique()}")

## 6️⃣ BrAPI Adapter Metadata

Inspect metadata schema (9 fields for sacred geometry alignment).

In [ ]:
# Initialize adapter
adapter = BrAPIAdapter(version='2.1')

# View default metadata
print("Default 9-field metadata schema:")
print(json.dumps(adapter.metadata, indent=2, default=str))

In [ ]:
# Custom metadata with harmonic seeding
custom_adapter = BrAPIAdapter(
    version='2.1',
    metadata={
        'timestamp': '2025-01-15T00:00:00Z',
        'source': 'C.C.R.O.P-PhenoHunt',
        'version': '3.0.0',
        'brapi_version': '2.1',
        'data_license': 'CC-BY-4.0',
        'contact': 'https://github.com/Hosuay/C.C.R.O.P-PhenoHunt',
        'citation': 'Hosuay et al. (2025)',
        'sacred_geometry_seed': 369,  # Harmonic seed
        'reproducibility_hash': 'sha256:abc123...'  # Data hash
    }
)

print("Custom metadata with harmonic seeding (369):")
print(json.dumps(custom_adapter.metadata, indent=2))

## 7️⃣ CLI Usage Examples

BrAPI adapter can also be used from command line.

In [ ]:
%%bash
# Import BrAPI JSON to CSV
# python -m src.io.brapi_adapter import --in data_examples/brapi/sample_traits.json --out traits.csv

# Export predictions to BrAPI JSON
# python -m src.io.brapi_adapter export --in predictions.csv --out predictions.json --study-id STUDY_001

# Validate BrAPI JSON
# python -m src.io.brapi_adapter validate data_examples/brapi/sample_traits.json

echo "CLI commands ready (uncomment to run)"

## Summary

✅ **BrAPI v2.1 Compliance**: Full support for Breedbase/BMS integration  
✅ **Import/Export**: Seamless data exchange with breeding platforms  
✅ **Validation**: Automatic structure verification  
✅ **Metadata**: 9-field schema for reproducibility  
✅ **Sacred Geometry**: Harmonic seeding integration (369)  

### Next Steps

1. Connect to Breedbase API for live data import
2. Integrate with PhytoOracle phenomics pipeline
3. Add genotype data exchange (BrAPI Genotyping endpoints)
4. Implement trial management metadata

---

**C.C.R.O.P-PhenoHunt** | Version 3.0.0 | Sacred Geometry ✨ 369 ✨